In [1]:
import os, sys

Mounted at /content/drive


In [2]:
import pandas as pd
import plotly

In [3]:
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
import plotly.express as px
from sklearn.cluster import KMeans

In [4]:
DATA_DIR = './data'
train = os.path.join(DATA_DIR, 'train.csv')
test = os.path.join(DATA_DIR, 'test.csv')

train_df, test_df = pd.read_csv(train), pd.read_csv(test)

## Check null data

In [5]:
train_df.isnull().any()

rent_approval_date     False
town                   False
block                  False
street_name            False
flat_type              False
flat_model             False
floor_area_sqm         False
furnished              False
lease_commence_date    False
latitude               False
longitude              False
elevation              False
subzone                False
planning_area          False
region                 False
monthly_rent           False
dtype: bool

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   rent_approval_date   60000 non-null  object 
 1   town                 60000 non-null  object 
 2   block                60000 non-null  object 
 3   street_name          60000 non-null  object 
 4   flat_type            60000 non-null  object 
 5   flat_model           60000 non-null  object 
 6   floor_area_sqm       60000 non-null  float64
 7   furnished            60000 non-null  object 
 8   lease_commence_date  60000 non-null  int64  
 9   latitude             60000 non-null  float64
 10  longitude            60000 non-null  float64
 11  elevation            60000 non-null  float64
 12  subzone              60000 non-null  object 
 13  planning_area        60000 non-null  object 
 14  region               60000 non-null  object 
 15  monthly_rent         60000 non-null 

In [7]:
train_df.furnished.unique()  # removed ?

array(['yes'], dtype=object)

## preprocessing

1. Convert rent_approval_date and lease_commence_data to datetime format
2. make other string entry to be lower case

In [48]:
approval_date_f = '%Y-%m'
commence_f = '%Y'

def clean_data(df):
    df = df.copy()
    string_attr = ['region', 'town', 'planning_area', 'subzone', 'town', 'block', 'street_name', 'flat_type', 'flat_model']

    for string_data in string_attr:
        df[string_data] = df[string_data].map(lambda x: x.lower())
    # df.rent_approval_date = df.rent_approval_date.map(lambda x: datetime.strptime(x, '%Y-%M').timestamp())
    # df.lease_commence_date = df.lease_commence_date.map(lambda x: datetime.strptime(str(x), '%Y').timestamp())
    df.flat_type = df.flat_type.map(lambda x: x if x.split(' ') == 1 else '-'.join(x.split(' '))) # normalize the flat type
    df = df.drop(columns=['furnished', 'elevation'])
    return df

def process_data(df):
  df = df.copy()
  df['age'] = df.apply(lambda x: (datetime.strptime(x['rent_approval_date'], approval_date_f) - datetime.strptime(str(x['lease_commence_date']), commence_f)).days, axis=1)
  return df

In [49]:
train_df_cleaned = clean_data(train_df)
train_df_cleaned = process_data(train_df_cleaned)
train_df_cleaned

,rent_approval_date,town,block,street_name,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,subzone,planning_area,region,monthly_rent,age
0,2021-09,jurong east,257,jurong east street 24,3-room,new generation,67.0,1983,1.344518,103.738630,yuhua east,jurong east,west region,1600,14123
1,2022-05,bedok,119,bedok north road,4-room,new generation,92.0,1978,1.330186,103.938717,bedok north,bedok,east region,2250,16191
2,2022-10,toa payoh,157,lorong 1 toa payoh,3-room,improved,67.0,1971,1.332242,103.845643,toa payoh central,toa payoh,central region,1900,18901
3,2021-08,pasir ris,250,pasir ris street 21,executive,apartment,149.0,1993,1.370239,103.962894,pasir ris drive,pasir ris,east region,2850,10439
4,2022-11,kallang/whampoa,34,whampoa west,3-room,improved,68.0,1972,1.320502,103.863341,bendemeer,kallang,central region,2100,18567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,2021-09,ang mo kio,441,ang mo kio avenue 10,3-room,new generation,67.0,1979,1.366050,103.854168,chong boon,ang mo kio,north-east region,2200,15584
59996,2023-04,bukit merah,95a,henderson road,4-room,model a,83.0,2019,1.286493,103.821434,henderson hill,bukit merah,central region,4100,1551
59997,2022-06,tampines,862a,tampines street 83,5-room,improved,122.0,1988,1.355064,103.936507,tampines west,tampines,east region,2250,12570
59998,2023-01,bedok,67,bedok sth avenue 3,5-room,standard,123.0,1977,1.318974,103.944076,bedok south,bedok,east region,4700,16801


### Check column values

In [10]:
train_df_cleaned['flat_model'].unique(), len(train_df_cleaned['flat_model'].unique())

(array(['new generation', 'improved', 'apartment', 'premium apartment',
        'simplified', 'model a', 'standard', 'dbss', 'model a2',
        'maisonette', 'model a-maisonette', 'type s1', 'adjoined flat',
        'type s2', 'premium apartment loft', '2-room',
        'premium maisonette', 'terrace', '3gen'], dtype=object),
 19)

In [11]:
train_df_cleaned['flat_type'].unique(), len(train_df_cleaned['flat_type'].unique())

(array(['3-room', '4-room', 'executive', '5-room', '2-room'], dtype=object), 5)

In [12]:
train_df_cleaned['type_model'] = train_df_cleaned['flat_model'] + ' ' + train_df_cleaned['flat_type']
train_df_cleaned['type_model'].unique(), len(train_df_cleaned['type_model'].unique())

(array(['new generation 3-room', 'new generation 4-room',
        'improved 3-room', 'apartment executive',
        'premium apartment executive', 'premium apartment 5-room',
        'simplified 4-room', 'model a 4-room', 'improved 5-room',
        'model a 3-room', 'standard 3-room', 'simplified 3-room',
        'dbss 4-room', 'model a2 4-room', 'improved 4-room',
        'premium apartment 4-room', 'model a 2-room',
        'maisonette executive', 'improved 2-room', 'standard 5-room',
        'model a 5-room', 'premium apartment 3-room', 'dbss 3-room',
        'dbss 5-room', 'model a-maisonette 5-room', 'type s1 4-room',
        'adjoined flat executive', 'type s2 5-room', 'standard 4-room',
        'adjoined flat 5-room', 'premium apartment loft 4-room',
        'standard 2-room', 'adjoined flat 4-room', '2-room 2-room',
        'premium maisonette executive', 'premium apartment 2-room',
        'terrace 3-room', 'terrace 4-room', '3gen 5-room'], dtype=object),
 39)

In [13]:
train_df_cleaned['psqm'] = train_df_cleaned['monthly_rent'] / train_df_cleaned['floor_area_sqm']

## Average Price across time (overall)

In [14]:
pd.options.plotting.backend = "plotly"

In [15]:
trained_df_group_avg = train_df_cleaned.groupby('rent_approval_date').mean(numeric_only=True).reset_index()

In [16]:
trained_df_group_avg.plot.line(x='rent_approval_date', y='psqm')


## Visualize the location cluster (First Glance)

In [17]:
data = train_df[['latitude', 'longitude']].to_dict(orient='list')
data.keys()

dict_keys(['latitude', 'longitude'])

In [18]:
def cluster_location(data, n_clusters):

    df = pd.DataFrame(data)

    # Number of clusters (change this based on your requirement)

    # KMeans clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(df[['latitude', 'longitude']])
    df['cluster'] = kmeans.labels_

    # Plotting on map using Plotly with Mapbox
    fig = px.scatter_mapbox(df,
                            lat='latitude',
                            lon='longitude',
                            color='cluster',
                            zoom=10,  # Adjust zoom level as needed
                            mapbox_style="open-street-map",  # Use open-street-map style
                            color_continuous_scale=px.colors.sequential.Plasma,
                            title="KMeans Clustering of rented house")

    fig.show()


In [19]:
cluster_location(data=data, n_clusters=6)

Output hidden; open in https://colab.research.google.com to view.

## Analysis For different type

In [21]:
train_df_cleaned.columns

Index(['rent_approval_date', 'town', 'block', 'street_name', 'flat_type',
       'flat_model', 'floor_area_sqm', 'lease_commence_date', 'latitude',
       'longitude', 'subzone', 'planning_area', 'region', 'monthly_rent',
       'type_model', 'psqm'],
      dtype='object')

In [34]:
def plot_psqm_trend(target_attr):
  trained_df_group_avg = train_df_cleaned.groupby(['rent_approval_date', target_attr]).mean(numeric_only=True).reset_index()
  return trained_df_group_avg.pivot(index='rent_approval_date', columns=target_attr, values='psqm').plot.line(title=f'{target_attr} per squre meter price', yaxis_title='')

In [35]:
plot_psqm_trend('region').show()

In [37]:
plot_psqm_trend('town').show()

Even though region price and overall price presents a strong raising singal, however, the price with town has more occilation. It may suggest that some other factor during the time period affecting the price as well.